# Primer Design Script
## Given a consensus sequence, this script will examine and present the possible primers with given length

In [7]:
import os
import Bio
from Bio import SeqIO
import re
import pandas as pd

In [51]:
import numpy as np

In [8]:
os.chdir("/home/ekinbasar/Spring_21-22/bioeksen_biotech/consensus_genome/HAV")

In [157]:
seq_object = SeqIO.read("hav_95_cons.fasta", "fasta")

In [158]:
sequence = str(seq_object.seq)

In [159]:
# sequence = str(sequence[0:500]) # -> short sequence extraction as a sample

## My Approach

Instead of using the previous approach, it might be beneficial to examine the string data with basic python functions.


-> Following cells use split(), index(), append(), enumarate() functions and list comprehension 

In [11]:
length_cut_off = 18 # decide the minimum length we are looking for in the sequences without N
                    # the sequences shorter than this length are ignored

matches = sequence.split("N")
matches=[x for x in matches if len(x)>=length_cut_off]#decide the shortest length that to be included to the list

In [12]:
# get the lengths of the matching substrings from the "matches" list
length_list = []

for sequences in matches:
    length_list.append(len(sequences))

In [13]:
length_list

[29,
 50,
 39,
 61,
 33,
 21,
 23,
 69,
 24,
 19,
 27,
 19,
 27,
 23,
 26,
 26,
 20,
 32,
 26,
 23,
 20,
 31,
 35,
 23,
 20,
 20,
 23,
 26,
 29,
 23,
 26,
 29,
 25,
 23,
 31,
 24,
 19,
 26,
 26,
 24,
 20,
 26,
 20,
 26,
 21,
 20,
 23,
 29,
 26,
 20,
 29,
 83,
 32,
 20,
 20,
 20,
 24,
 23,
 23,
 41,
 29,
 35,
 50,
 29,
 28,
 20,
 23,
 18,
 20,
 44,
 25,
 75]

In [14]:
# get the indexes of the matching substrings from the "matches" list
index_list = []
for i in matches:
    index = sequence.index(i)
    index_list.append(index)

In [15]:
# the list that stores the indexes of the matching substrings - starting position of the sequence
index_list

[0,
 47,
 135,
 222,
 290,
 342,
 388,
 443,
 513,
 546,
 573,
 602,
 649,
 755,
 779,
 1028,
 1064,
 1091,
 1514,
 1613,
 2144,
 2331,
 2363,
 2399,
 2438,
 2606,
 2672,
 2894,
 2948,
 3062,
 3263,
 3290,
 3738,
 3764,
 3867,
 4412,
 4613,
 4696,
 4786,
 4813,
 4852,
 4948,
 5059,
 5371,
 5452,
 5680,
 5773,
 5797,
 5872,
 5908,
 5935,
 5965,
 6049,
 6184,
 6262,
 6409,
 6451,
 6544,
 6598,
 6649,
 6703,
 6778,
 6832,
 6937,
 7121,
 7204,
 7237,
 7296,
 7315,
 7364,
 7409,
 7435]

In [16]:
# the distance between consecutive elements 
# -> the distance between the elemenet and the next one (sequence-wise sorting)  
distance_list = [x - index_list[i - 1] for i, x in enumerate(index_list)][1:]
# the 0 is added at the end because the last sequence has no subsequent sequence, its the last one :)
distance_list.append(0)

In [17]:
distance_list

[47,
 88,
 87,
 68,
 52,
 46,
 55,
 70,
 33,
 27,
 29,
 47,
 106,
 24,
 249,
 36,
 27,
 423,
 99,
 531,
 187,
 32,
 36,
 39,
 168,
 66,
 222,
 54,
 114,
 201,
 27,
 448,
 26,
 103,
 545,
 201,
 83,
 90,
 27,
 39,
 96,
 111,
 312,
 81,
 228,
 93,
 24,
 75,
 36,
 27,
 30,
 84,
 135,
 78,
 147,
 42,
 93,
 54,
 51,
 54,
 75,
 54,
 105,
 184,
 83,
 33,
 59,
 19,
 49,
 45,
 26,
 0]

### Tm Calculation

From a websourse, the equation is suggested for sequences longer than 14 bases:

        Tm= 64.9 +41*(yG+zC-16.4)/(wA+xT+yG+zC)
        
        
        
    Example:

            ATAGCGTTGCTGCTCCGAAATT

            -> 64.9 + 41*(10-16.4)/22 = 52.97272727272728
            
            Tool predicts 53

In [35]:
def tm_calculator(sequence):
   
    g_c_count = sequence.count("C") + sequence.count("G")

    tm = "%.2f" % (64.9 + 41 * (g_c_count - 16.4) / len(sequence)) # 2 numbers are allowed after the dot
    
    return tm

In [36]:
sequence = "ATAGCGTTGCTGCTCCGAAATT"
tm_calculator(sequence)

'52.97'

In [37]:
tm_list = []

for i in matches:
    # print(tm_calculator(i))
    tm_list.append(tm_calculator(i))

In [38]:
tm_list

['64.33',
 '73.59',
 '61.33',
 '81.43',
 '64.40',
 '52.40',
 '58.84',
 '74.76',
 '57.38',
 '53.25',
 '61.26',
 '44.62',
 '61.26',
 '55.27',
 '59.54',
 '54.81',
 '45.63',
 '57.98',
 '50.08',
 '49.93',
 '43.58',
 '64.37',
 '59.75',
 '51.71',
 '45.63',
 '47.68',
 '58.84',
 '50.08',
 '55.85',
 '53.49',
 '51.65',
 '61.51',
 '51.12',
 '44.58',
 '56.44',
 '52.26',
 '42.46',
 '50.08',
 '53.23',
 '47.13',
 '43.58',
 '50.08',
 '49.73',
 '56.38',
 '44.60',
 '47.68',
 '57.06',
 '65.75',
 '54.81',
 '43.58',
 '51.61',
 '71.12',
 '59.26',
 '41.53',
 '41.53',
 '43.58',
 '52.26',
 '55.27',
 '53.49',
 '60.50',
 '55.85',
 '57.40',
 '69.49',
 '55.85',
 '55.53',
 '45.63',
 '51.71',
 '36.66',
 '43.58',
 '63.60',
 '51.12',
 '61.40']

        Create a dataframe to see how it's going

In [150]:
sequence_dictionary = {"Matches":matches,"Lengths":length_list,"Start Position":index_list,"Tm":tm_list, "Distance(between consecutive start points)[Subsequent - First]":distance_list}

In [151]:
sequence_df = pd.DataFrame(sequence_dictionary)

In [152]:
sequence_df["Tm/Length"] = pd.to_numeric(sequence_df["Tm"]) / sequence_df["Lengths"] # add the Tm / Length column

In [153]:
sequence_df

,Matches,Lengths,Start Position,Tm,Distance(between consecutive start points)[Subsequent - First],Tm/Length
0,TTCAAGAGGGGTCTCCGGAGTTTTCCGGA,29,0,64.33,47,2.218276
1,TGGTGAGGGGACTTGATACCTCACCGCCGTTTGCCTAGGCTATAGG...,50,47,73.59,88,1.471800
2,TTGTTTGTAAATATTAATTCCTGCAGGTTCAGGGTTCTT,39,135,61.33,87,1.572564
3,CTTTCTTCCAGGGCTCTCCCCTTGCCCTAGGCTCTGGCCGTTGCGC...,61,222,81.43,68,1.334918
4,TAGCATGGAGCTGTAGGAGTCTAAATTGGGGAC,33,290,64.40,52,1.951515
...,...,...,...,...,...,...
67,TGAGTTTTATCAGAAATT,18,7296,36.66,19,2.036667
68,TATTATTTTGTTCAGTCCTG,20,7315,43.58,49,2.179000
69,CTTAAATCTTATGATTGGTGGAGAATGAGATTTTATGACCAGTG,44,7364,63.60,45,1.445455
70,TTCATTTGTGACCTTTCATGATTTG,25,7409,51.12,26,2.044800


    Until now, we have the lists of:
        Matching Sequences
        Lengths
        Start Position (index) 
        Tm
        Distance from the starting point of one match to the next match's starting point 
        Tm / Length ration of the sequences
        
        Next, analyse the combined sequences pair-wise and as triad :)

### Get the combined two sequences that satisfy the conditions

In [ ]:
# get two combined sequences that satisfy the conditions

# this cell requires a clean up + check the Tm's of the sequences -> Tm bottomline is 50; drop if less

for i in range(len(distance_list)-1):

    distance = distance_list[i]
    
    if distance < 200: 
        
        if (tm_list[i],tm_list[i+1]) > 50:
            
        starting_sequence_index = index_list[i]
        starting_sequence = matches[distance_list.index(distance)]
        subsequent_sequence = matches[i+1]
        subsequent_sequence_length = length_list[int(i)+1]
        partition_length = distance + subsequent_sequence_length
        combined_sequence = sequence[starting_sequence_index:(starting_sequence_index+partition_length)]


### Next, get the combined three sequences that satisfy the conditions 

      # We can specify an amplicon size upper limit and display the ones that satisfy.
      
      # Similarly, we can change the distance (between starting points) condition
      
      # I added the Tm lower limit condition from the excel sheet (Şükrü Bey - Design)

In [119]:
distance_list_1 = [x - index_list[i - 2] for i, x in enumerate(index_list)][1:]
distance_list_1.pop(0)
distance_list_1.append(0)
distance_list_1.append(0)

# two zeros added at the end of the list
# this list works like that: It has the distance between the index(starting point) of the corresponding sequence 
# and the sequence two index later

# Yani ilk sıradaki element, aslında üçüncünün birinciden farkı, 
# o yüzden son iki element 0 çünkü onlardan 2 sonraki elementler diye bir şey yok

In [175]:
# get three combined sequences that satisfy the conditions

for i in range(len(distance_list_1)-2):

    distance = distance_list_1[i]
    
    
    if distance < 200:     
        
        if float(tm_list[i]) and float(tm_list[i+1]) and float(tm_list[i+2]) > 50:
            starting_sequence_index = index_list[i]
    
            third_sequence_length = length_list[int(i)+2]
            partition_length = distance + third_sequence_length 
            combined_sequence = sequence[starting_sequence_index:(starting_sequence_index+partition_length)]
            
            print("The Tm's of the sequences are (respectively):",tm_list[i],tm_list[i+1],tm_list[i+2])
            print("The length of the combined sequences (amplicon size) is:", partition_length)
            print(combined_sequence,"\n")

        
        
        # Partition Length -> len(p)
        # len(p) = Index(3rd) - Index(1st) + len(3rd)

The Tm's of the sequences are (respectively): 64.33 73.59 61.33
The length of the combined sequences (amplicon size) is: 174
TTCAAGAGGGGTCTCCGGAGTTTTCCGGANCCCCTCTTGGAAGTCCNTGGTGAGGGGACTTGATACCTCACCGCCGTTTGCCTAGGCTATAGGCTAANTTTCCCTTTCCCTGTCCNTNNCNNATTNCCNTTTGTNTTGTTTGTAAATATTAATTCCTGCAGGTTCAGGGTTCTT 

The Tm's of the sequences are (respectively): 73.59 61.33 81.43
The length of the combined sequences (amplicon size) is: 236
TGGTGAGGGGACTTGATACCTCACCGCCGTTTGCCTAGGCTATAGGCTAANTTTCCCTTTCCCTGTCCNTNNCNNATTNCCNTTTGTNTTGTTTGTAAATATTAATTCCTGCAGGTTCAGGGTTCTTNAATCTGTTTCTCTATAANAACACTCANNTTTTNACGCTTTCTGTCTNCTTTCTTCCAGGGCTCTCCCCTTGCCCTAGGCTCTGGCCGTTGCGCCCGGCGGGGTCAACT 

The Tm's of the sequences are (respectively): 61.33 81.43 64.40
The length of the combined sequences (amplicon size) is: 188
TTGTTTGTAAATATTAATTCCTGCAGGTTCAGGGTTCTTNAATCTGTTTCTCTATAANAACACTCANNTTTTNACGCTTTCTGTCTNCTTTCTTCCAGGGCTCTCCCCTTGCCCTAGGCTCTGGCCGTTGCGCCCGGCGGGGTCAACTNCATGANTAGCATGGAGCTGTAGGAGTCTAAATTGGGGAC 

The Tm's of the se

## A nice report format for the consecutive sequence analysis

In [169]:
# This for loop it to display the outcomes for two consecutive sequences that satisfy the conditions


for i in range(len(distance_list)-1):

    distance = distance_list[i]
    
    if distance < 200:
        
        if float(tm_list[i]) and float(tm_list[i+1]) > 50:
            
            starting_sequence_index = index_list[i]

            starting_sequence = matches[distance_list.index(distance)]

            subsequent_sequence = matches[i+1]

            subsequent_sequence_length = length_list[int(i)+1]

            partition_length = distance + subsequent_sequence_length

            combined_sequence = sequence[starting_sequence_index:(starting_sequence_index+partition_length)]
            # the combined sequence is acquired by indexing the whole sequence with the starting position of the first 
            # sequence and the sum up of the starting position (indexed from the whole sequence)of the first sequence 
            # and the length of the second sequence


            print("The first sequence's index is:",starting_sequence_index)
            print("The first sequence is:",starting_sequence)
            print("The Tm of the first sequence is:", tm_list[i])
            print("The length of the first sequence is:",length_list[i])
            print("The starting point distance between first and the second sequence is:",distance)
            print("The second sequence is:",subsequent_sequence)
            print("The Tm of the second sequence is:", tm_list[i])
            print("The length of the second sequence is:",subsequent_sequence_length)
            print("The length of the two sequences combined is:",partition_length)
            print("The N count for the combined sequence is:",combined_sequence.count("N"))
            print("The combined sequence for two consecutive sequences from the consensus genome is:\n", combined_sequence,"\n")
            


        

The first sequence's index is: 0
The first sequence is: TTCAAGAGGGGTCTCCGGAGTTTTCCGGA
The Tm of the first sequence is: 64.33
The length of the first sequence is: 29
The starting point distance between first and the second sequence is: 47
The second sequence is: TGGTGAGGGGACTTGATACCTCACCGCCGTTTGCCTAGGCTATAGGCTAA
The Tm of the second sequence is: 64.33
The length of the second sequence is: 50
The length of the two sequences combined is: 97
The N count for the combined sequence is: 2
The combined sequence for two consecutive sequences from the consensus genome is:
 TTCAAGAGGGGTCTCCGGAGTTTTCCGGANCCCCTCTTGGAAGTCCNTGGTGAGGGGACTTGATACCTCACCGCCGTTTGCCTAGGCTATAGGCTAA 

The first sequence's index is: 47
The first sequence is: TGGTGAGGGGACTTGATACCTCACCGCCGTTTGCCTAGGCTATAGGCTAA
The Tm of the first sequence is: 73.59
The length of the first sequence is: 50
The starting point distance between first and the second sequence is: 88
The second sequence is: TTGTTTGTAAATATTAATTCCTGCAGGTTCAGGGTTCTT
The Tm 

## Previous Algorithm (from Excel sheet)

In this step we created three lists that store information about the N number. This allows us to examine the conservation status of sequences in our consensus sequence window by window. 

In [193]:
"""zero_n_window_list = []

zero_n_list = []

increased_index_list = []


for i in range(len(sequence)): 
    # here, I first used (len(sequence) - 18) to prevent any possible indexing problem 
    # but then checkedimport numpy as np
 with whole length, did not give rise to any problem, consider this later

    window = sequence[i:i+18]
    
    n_count = window.count("N")
    
    if n_count != 0:
        
        zero_n_window_list.append(0)
        
        zero_n_list.append(0)
        
        increased_index_list.append(0)
        
    else:
        zero_n_window_list.append(window)
        
        zero_n_list.append(1)

        if len(increased_index_list) != 0:

            last_element = increased_index_list[-1]

            if last_element == 0:
                increased_index_list.append(1)
            elif last_element != 0:
                increased_index_list.append(last_element + 1)

        else:
            increased_index_list.append(1)
        
        
print(zero_n_window_list[11])
print(increased_index_list[11])
print(zero_n_list[0:13])
"""

'zero_n_window_list = []\n\nzero_n_list = []\n\nincreased_index_list = []\n\n\nfor i in range(len(sequence)): \n    # here, I first used (len(sequence) - 18) to prevent any possible indexing problem \n    # but then checked with whole length, did not give rise to any problem, consider this later\n\n    window = sequence[i:i+18]\n    \n    n_count = window.count("N")\n    \n    if n_count != 0:\n        \n        zero_n_window_list.append(0)\n        \n        zero_n_list.append(0)\n        \n        increased_index_list.append(0)\n        \n    else:\n        zero_n_window_list.append(window)\n        \n        zero_n_list.append(1)\n\n        if len(increased_index_list) != 0:\n\n            last_element = increased_index_list[-1]\n\n            if last_element == 0:\n                increased_index_list.append(1)\n            elif last_element != 0:\n                increased_index_list.append(last_element + 1)\n\n        else:\n            increased_index_list.append(1)\n        \n 

regex did not worked well, check it later

In the following cell, I used regex to extract strings without N but surrounded by N's. Thus, "N" character is added to the beginning of the sequence just to ensure any possible matching string at the start are also recovered. 

In [190]:
# regex solution - might be less memory consuming and faster

# string = str("N" + sequence) # -> has problems, does not match all substrings 

# matches = re.findall(r'N((?:(?!N).)*?)N', string)
# matches = re.findall(r'N(.*?)N', string)
# print(len(matches),matches)